<a href="https://colab.research.google.com/github/SilverFoxMedia-Tesis/IA_02/blob/main/Random_Forest_Model_Flby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
# np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

datos = pd.read_csv("/dataset_final_v4.csv")
print(datos.head())

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

input_columns = [
    "Deficit",
    "desayuno_carbs",
    "desayuno_proten",
    "desayuno_grasas",
    "almuerzo_carbs",
    "almuerzo_proten",
    "almuerzo_grasas",
    "cena_carbs",
    "cena_proten",
    "cena_grasas"
]
output_columns = [
    "desayuno_1",
    "desayuno_2",
    "desayuno_3",
    "almuerzo_1",
    "almuerzo_2",
    "almuerzo_3",
    "cena_1",
    "cena_2"
]

filtered_df = datos[input_columns + output_columns].dropna()
output_mappings = {}
for col in output_columns:
    filtered_df[col], output_mappings[col] = filtered_df[col].factorize()
X = filtered_df[input_columns].values
y = filtered_df[output_columns].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = {col: RandomForestClassifier(n_estimators=100, random_state=42) for col in output_columns}

for idx, col in enumerate(output_columns):
    models[col].fit(X_train, y_train[:, idx])

predictions = np.zeros(y_test.shape)
for idx, col in enumerate(output_columns):
    predictions[:, idx] = models[col].predict(X_test)



   Gender  Age  Diabetes  Weight  Height  AbdominalCircumference  \
0       1   39       2.0    75.0   171.0                    95.6   
1       2   22       2.0    65.9   158.1                    89.5   
2       2   20       2.0    58.3   161.5                    82.9   
3       1   33       2.0    73.8   170.7                    93.7   
4       2   38       2.0    53.9   151.5                    74.9   

   PhysicalActivity      TMB  ClassificationIMC  Deficit  ...  \
0                 1  1749.00                  3  1399.20  ...   
1                 2  2027.48                  3  1621.98  ...   
2                 2  1949.09                  2  1559.27  ...   
3                 1  1735.08                  3  1388.06  ...   
4                 1  1297.93                  2  1038.34  ...   

              almuerzo_1            almuerzo_2          almuerzo_3  \
0                Ham as   Navy baked with pork          Cornflakes   
1                Herring        Raw 2 by 2 1/2         Rice 

In [2]:
# Selecciona un único dato de prueba (por ejemplo, el primer dato del conjunto de prueba)
single_test = X_test[0].reshape(1, -1)
true_labels = y_test[0]

# Predice para el único dato
single_predictions = []
for idx, col in enumerate(output_columns):
    single_predictions.append(models[col].predict(single_test)[0])

# Comparar predicciones con etiquetas reales
print("Predicciones:", single_predictions)
print("Etiquetas reales:", true_labels)

# Puedes decodificar las etiquetas usando los mapeos inversos para obtener los nombres de comida
decoded_predictions = [list(output_mappings[col])[pred] for col, pred in zip(output_columns, single_predictions)]
decoded_true_labels = [list(output_mappings[col])[true_label] for col, true_label in zip(output_columns, true_labels)]

print("Predicciones decodificadas:", decoded_predictions)
print("Etiquetas reales decodificadas:", decoded_true_labels)


Predicciones: [2, 2, 9, 14, 11, 16, 8, 7]
Etiquetas reales: [ 3  1 12 26  2 19  3 17]
Predicciones decodificadas: ['Processed cheese', 'Swiss', 'Raw red', 'Bacon', 'heated peas', 'Puffed wheat presweetened', 'Bouillon', 'Raw red']
Etiquetas reales decodificadas: ['Cream cheese', 'Eggs raw', 'Fresh raw figs', 'Salmon', 'Winter mashed', 'Wheat germ', 'Tomato soup', 'Raspberries']


In [3]:
import joblib

# Guardar los modelos en archivos individuales
for col, model in models.items():
    joblib.dump(model, f'{col}_model.pkl')

# Guarda también los mapeos para las etiquetas
joblib.dump(output_mappings, 'output_mappings.pkl')


['output_mappings.pkl']

In [4]:
from google.colab import files

# Descargar cada archivo individualmente
files.download('desayuno_1_model.pkl')
files.download('desayuno_2_model.pkl')
files.download('output_mappings.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import sklearn
print(sklearn.__version__)


1.2.2
